In [3]:
import numpy as np
import math
import json
from collections import Counter

In [4]:
def createDataSet():
    dataSet = [[0, 0, 0, 0, 'no'],         #数据集
                [0, 0, 0, 1, 'no'],
                [0, 1, 0, 1, 'yes'],
                [0, 1, 1, 0, 'yes'],
                [0, 0, 0, 0, 'no'],
                [1, 0, 0, 0, 'no'],
                [1, 0, 0, 1, 'no'],
                [1, 1, 1, 1, 'yes'],
                [1, 0, 1, 2, 'yes'],
                [1, 0, 1, 2, 'yes'],
                [2, 0, 1, 2, 'yes'],
                [2, 0, 1, 1, 'yes'],
                [2, 1, 0, 1, 'yes'],
                [2, 1, 0, 2, 'yes'],
                [2, 0, 0, 0, 'no']]
    labels = ['年龄', '有工作', '有自己的房子', '信贷情况']#分类属性
    return dataSet, labels                #返回数据集和分类属性



In [7]:
#计算香农熵
#data:分类的列表
def calcShannonEnt(data):
    cot=Counter(list(data))
    total=len(data)
    H=0
    for v in cot.values():
        H+=v/total*math.log(v/total,2)
    return -H


DataSet,Labels=createDataSet()
DataSet=np.array(DataSet)
calcShannonEnt(DataSet[:,-1])

0.9709505944546686

In [103]:
#判断是否非数值型
def is_Continuous_values(dataSet):
    flag=0
    for i in dataSet:
        try:
            if isinstance(float(i),float):
                flag=1
            else:
                return 0
        except:
            return 0
    return flag

In [104]:
#西瓜数据集测试
with open('watermelon2.txt','r') as f:
    lines=f.readlines()
    dataSet=[]
    for line in lines:
        dataSet.append(line.replace('\n','').split(','))
        
data=dataSet[1:]
data=np.array(data)[:,1:]
is_Continuous_values(data[:,-2])
# Continuous_values_analysis(data[:,[-3,-1]])

1

In [69]:
#连续值处理
def  Continuous_values_analysis(data):
    X=data[:,0].astype(float)
    X_temp=data[:,0].astype(float)
    Y=data[:,-1]
    length=len(X)
    T=[]
    X.sort()
    for ix in range(1,length):
        result=(X[ix]+X[ix-1])/2
        T.append(result)
    EntD=calcShannonEnt(Y)
    maxgain=0
    maxt=0
    for t in T:
        cate_list1=Y[np.where(X_temp>t)]
        cate_list2=Y[np.where(X_temp<t)]
        Ent1=calcShannonEnt(cate_list1)
        Ent2=calcShannonEnt(cate_list2)
        gain=EntD-len(cate_list1)/length*Ent1-len(cate_list2)/length*Ent2
        if gain>maxgain:
            list1,list2=np.where(X_temp>t),np.where(X_temp<t)
            maxgain=gain
            maxt=t
    return maxgain,maxt,list1,list2

In [21]:
#dataset:numpy对象
#return:返回特征增益的list
def chooseBestFeatureToSplit(dataSet):
    #计算信息增益，选择最优特征
    #1.划分特征
    #2.每列划分出不同类别
    #3.每个类别对应的Y特征数量
    row_len,column_len=dataSet.shape
    label_col=dataSet[:,-1]
    Hd=calcShannonEnt(label_col)
    gs=[]#用于存储对应特征的增益
    for col in range(0,column_len-1):
        features=dataSet[:,col]
        cot=Counter(features)
        g=0
        for cate,v in cot.items():
            #该特征下为cate的所有行
            cate_rows=np.where(features==cate)
            #某类在对应Y的香农熵
            Hi=calcShannonEnt(label_col[cate_rows])
            g+=v/row_len*Hi
        Gain=Hd-g#ID3算法
        gs.append(Gain)
    return gs
    
def chooseBestFeatureToSplit_index(dataSet):
    #计算信息增益，选择最优特征
    #1.划分特征
    #2.每列划分出不同类别
    #3.每个类别对应的Y特征数量
    row_len,column_len=dataSet.shape
    label_col=dataSet[:,-1]
    Hd=calcShannonEnt(label_col)
    gs=[]#用于存储对应特征的增益
    for col in range(0,column_len-1):
        features=dataSet[:,col]
#         if is_Continuous_values(features):
#             #是连续数值型
#             pass
        cot=Counter(features)
        g=0
        for cate,v in cot.items():
            #该特征下为cate的所有行
            cate_rows=np.where(features==cate)
            #某类在对应Y的香农熵
            Hi=calcShannonEnt(label_col[cate_rows])
            g+=v/row_len*Hi
        Gain=Hd-g
        gs.append(Gain)
    #ID3算法
    max_Ent=max(gs)
    index=gs.index(max_Ent)#返回最大信息增益的下标 
    #C4.5算法
    gs_array=np.array(gs)
    gs_mean=np.mean(gs_array)
    row_list=np.where(gs_array>gs_mean)
    gs_RTmean_array=gs_array[row_list]
    max_ix=index
    max_Gain_radio=0
    for num in list(gs_RTmean_array):
        ix=gs.index(num)
        IV=calcShannonEnt(dataSet[:,ix])
        Gain_radio=num/IV
        if Gain_radio>max_Gain_radio:
            max_ix=ix
            max_Gain_radio=Gain_radio
    index=max_ix
    return index

#CART算法
def Gini(dataSet):
    cate_v=Counter(list(dataSet))
    data_len=len(dataSet)
    S=0
    for k,v in cate_v.items():
        S+=(v/data_len)**2
    return 1-S

def chooseBestFeatureToSplit_index_Gini(dataSet):
    row_len,column_len=dataSet.shape
    labels=dataSet[:,-1]
    gini_list=[]
    for col in range(column_len-1):
        data_temp=dataSet[:,col]
        S=0
        for cate,v in Counter(data_temp).items():
            row_list=np.where(data_temp==cate)
            gi=Gini(labels[row_list])
            S+=(v/row_len)*gi
        gini_list.append(S)
        
    min_gini=min(gini_list)
    min_gini_index=gini_list.index(min_gini)
    return min_gini_index#返回最小gini值得下标
# Gini(data[:,3])
# chooseBestFeatureToSplit_index_Gini(data)
#chooseBestFeatureToSplit(data)


data,label=createDataSet()
data=np.array(data)
chooseBestFeatureToSplit_index(data)

2

In [223]:
2**3

8

In [22]:
#{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}['年龄', '有工作', '有自己的房子', '信贷情况']#分类属性
#dataSet:numpy对象，数据集
#labels：特征名['年龄', '有工作', '有自己的房子', '信贷情况']
#catelist：特征分类[['a','b'],['1','2']]
#
def createTree(dataSet,labels,cates_list):
    row_len,column_len=dataSet.shape
    data_cates=dataSet[:,-1]
    cates=Counter(data_cates).most_common()
    if len(cates)==1:
        #如果所有训练集都是一个分类,则设置为当前类别
        return cates[0][0]
    best_feature_index=chooseBestFeatureToSplit_index(dataSet)
#    best_feature_index=chooseBestFeatureToSplit_index_Gini(dataSet)
#     sort_gs=sorted(gs,reverse=True)
#     best_feature_max_Ent=sort_gs[0]
#     best_feature_index=gs.index(best_feature_max_Ent)
    best_label=labels[best_feature_index]
    best_feature_array=dataSet[:,best_feature_index]
    #切割datsSet，labels
    split_dataSets=np.delete(dataSet,best_feature_index,axis=1)
    split_labels=labels.copy()
    split_labels.pop(best_feature_index)
    split_cates_list=cates_list.copy()
    split_cates_list.pop(best_feature_index)
    #
    tree={best_label:{}}
    for best_feature_cate in cates_list[best_feature_index]:
        cate_row_indexs=np.where(best_feature_array==best_feature_cate)#对应类别的行标
        split_dataSet=split_dataSets[list(cate_row_indexs[0]),:]
        if split_dataSet.size==0:
            #该分类的数据集为空，则设置为其父结点的最大分类
            tree[best_label][best_feature_cate]=cates[0][0]
        else:
            tree[best_label][best_feature_cate]=createTree(split_dataSet,split_labels,split_cates_list) 
    return tree


#得到每一个特征的分类:[['a','b'],['0','1'],['0','1','2']]
def Elem2cates(dataSet):
    row,col=dataSet.shape
    cate_list=[]
    for j in range(col):
        cates=[]
        for cate in set(dataSet[:,j]):
            cates.append(cate)
        cate_list.append(cates)
    return cate_list

#存入树
def storeTree(tree,filename):
    with open(filename,'w') as f:
        f.write(json.dumps(tree))

#读取树
def loadTree(filename):
    with open(filename,'r') as f:
        tree=f.read()
    return json.loads(tree)

#转换数据，并创建和存储树
def Data2Tree(data,label,filename='tree.txt'):
    data=np.array(data)
    cates_list=Elem2cates(data)
    tree=createTree(data,label,cates_list)
    storeTree(tree,filename)
    print('成功生成并存储决策树：',tree)
# loadTree('tree.txt')    

data,label=createDataSet()
Data2Tree(data,label)


成功生成并存储决策树： {'有自己的房子': {'0': {'有工作': {'0': 'no', '1': 'yes'}}, '1': 'yes'}}


In [68]:
#西瓜数据集测试
with open('watermelon2.txt','r') as f:
    lines=f.readlines()
    dataSet=[]
    for line in lines:
        dataSet.append(line.replace('\n','').split(','))
        
data=dataSet[1:]
data=np.array(data)[:,1:]
Continuous_values_analysis(data[:,[-3,-1]])
# X=data[:,-3].astype(float)
# X_temp=data[:,-3].astype(float)
# Y=data[:,-1]
# length=len(X)
# T=[]
# X.sort()
# for ix in range(1,length):
#     result=(X[ix]+X[ix-1])/2
#     T.append(result)
# EntD=calcShannonEnt(Y)
# maxgain=0
# maxt=0
# for t in T:
#     cate_list1=Y[np.where(X_temp>t)]
#     cate_list2=Y[np.where(X_temp<t)]
#     Ent1=calcShannonEnt(cate_list1)
#     Ent2=calcShannonEnt(cate_list2)
#     gain=EntD-len(cate_list1)/length*Ent1-len(cate_list2)/length*Ent2
#     if gain>maxgain:
#         maxgain=gain
#         maxt=t

# labels=dataSet[0][1:]
# data=dataSet[1:]
# data=np.array(data)[:,1:]
# Data2Tree(data,labels)


(0.2624392604045631,
 0.3815,
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 12, 13, 15, 16], dtype=int64),),
 (array([ 9, 10, 11, 14], dtype=int64),))

In [23]:
def classify(tree,label,testvec):
    #isinstance (a,int)
    while isinstance(tree,dict):
        #取标签名
        items=list(tree.items())[0]
        feature_index=label.index(items[0])
        value=str(testvec[feature_index])
        tree=items[1][value]
    if tree=='yes':
        print('放贷')
    elif tree=='no':
        print('不放贷')
        
featLabels=['有自己的房子','有工作']
vec=[0,1]
tree=loadTree('tree.txt')
classify(tree,featLabels,vec)

放贷


In [241]:
with open('lenses.txt','r') as f:
    #print(f.readlines())
    data=[]
    for line in f.readlines():
        data.append(line.replace('\n','').split('\t'))

labels=['age','prescript','astigmatic','tearRate']
Data2Tree(data,labels)

成功生成并存储决策树： {'tearRate': {'normal': {'astigmatic': {'yes': {'prescript': {'hyper': {'age': {'young': 'hard', 'pre': 'no lenses', 'presbyopic': 'no lenses'}}, 'myope': 'hard'}}, 'no': {'age': {'young': 'soft', 'pre': 'soft', 'presbyopic': {'prescript': {'hyper': 'soft', 'myope': 'no lenses'}}}}}}, 'reduced': 'no lenses'}}


In [120]:
data,label=createDataSet()
data=np.array(data)
data1=data[:,0]
b=np.where(data1=='p')
c=data[b,:]
c.size
for i in set(data1):
    print(i)

# calcShannonEnt(data[:,-1][b])


1
0
2


In [178]:
a={'有自己的房子': {'1': 'yes', '0': {'有工作': {'1': 'yes', '0': 'no'}}}}
list(a.items())[0][1]


{'1': 'yes', '0': {'有工作': {'1': 'yes', '0': 'no'}}}

In [131]:
data,label=createDataSet()
data=np.array(data)
gs=chooseBestFeatureToSplit(data)
print(gs)
# gs=sorted(gs,reverse=True)
np.delete(data,4,axis=1)
print(data[[1,2],:])

[0.08300749985576883, 0.32365019815155627, 0.4199730940219749, 0.36298956253708536]
[['0' '0' '0' '1' 'no']
 ['0' '1' '0' '1' 'yes']]
